In [1]:
!pip install nltk
!pip install xgboost
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
#from wordcloud import WordCloud
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Lasso
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

# Load your data (replace with your actual data loading)
# Assuming your data is in a CSV file named 'data.csv'
import pandas as pd
new_columns=pd.read_csv('../data/neiss_10p_sample_new_columns.csv')
data_10P = pd.read_csv('../data/neiss_10p_sample.csv')

# sematic=pd.read_csv('../data/10P_sematic_bert.csv')
embedding=pd.read_csv('../data/gist_embedding_10p_v0.csv')

#embedding_2=pd.read_csv('../xwang3306/aliba_embedding_10p.csv')

[nltk_data] Downloading package punkt to /home/eric/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/eric/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/eric/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/eric/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data=data_10P.merge(new_columns,how='inner',on='CPSC_Case_Number').merge(embedding,how='inner',on='CPSC_Case_Number').reset_index(drop=True)
data.head(5)

,Unnamed: 0,CPSC_Case_Number,Treatment_Date,Age,Sex,Race,Other_Race,Hispanic,Body_Part,Diagnosis,...,374,375,376,377,378,379,380,381,382,383
0,0,221032332,2022-09-24,14,1,0,0,0.0,34,71,...,0.034040,-0.036666,0.008797,0.024984,-0.048995,0.002708,-0.020119,-0.058474,-0.022191,0.038710
1,1,181109464,2018-10-30,28,1,1,0,0.0,79,71,...,0.024239,-0.000142,0.007269,-0.015545,-0.040856,-0.042023,-0.050380,0.008760,-0.002266,0.043031
2,2,210103105,2020-10-24,35,1,0,0,0.0,30,53,...,0.060739,0.014632,0.022998,-0.055498,-0.040177,0.024382,-0.086975,-0.045614,-0.022894,-0.013169
3,3,161157997,2016-11-15,214,2,0,0,0.0,76,53,...,0.027231,-0.016303,-0.013848,-0.046165,-0.034185,0.006101,-0.041465,-0.041030,0.040551,0.013185
4,4,181107411,2018-10-21,4,1,0,0,0.0,92,72,...,0.019272,-0.061428,0.017961,0.035712,0.021475,0.038897,-0.028660,0.035182,-0.031062,-0.006973


In [3]:
data.columns

Index(['Unnamed: 0', 'CPSC_Case_Number', 'Treatment_Date', 'Age', 'Sex',
       'Race', 'Other_Race', 'Hispanic', 'Body_Part', 'Diagnosis',
       ...
       '374', '375', '376', '377', '378', '379', '380', '381', '382', '383'],
      dtype='object', length=416)

In [4]:
import numpy as np
data['Disposition_recode']=np.nan
data.loc[((data['Disposition']==1)), 'Disposition_recode'] = 0
data.loc[((data['Disposition']==2)), 'Disposition_recode'] = 1
data.loc[((data['Disposition']==4)), 'Disposition_recode'] = 2
data.loc[((data['Disposition']==5)), 'Disposition_recode'] = 3
data.loc[((data['Disposition']==8)), 'Disposition_recode'] = 4
data=data[data['Disposition_recode'].notna()]

data['Disposition_recode_2']=0
data.loc[((data['Disposition_recode']>0)), 'Disposition_recode_2'] = 1

In [5]:
data=data[(data['Body_Part']!=0) & (data['Body_Part']!=84) & (data['Body_Part']!=85) & (data['Body_Part']!=86) & (data['Body_Part']!=87)]
data['Disposition_recode'].value_counts()
data['Disposition_recode_2'].value_counts() 

Disposition_recode_2
0    296227
1     32874
Name: count, dtype: int64

In [6]:
bdpt_dict={}
bdpt_dict[0]='INTERNAL'
bdpt_dict[30]='SHOULDER'
bdpt_dict[31]='UPPERTRUNK'
bdpt_dict[32]='ELBOW'
bdpt_dict[33]='LOWERARM'
bdpt_dict[34]='WRIST'
bdpt_dict[35]='KNEE'
bdpt_dict[36]='LOWERLEG'
bdpt_dict[37]='ANKLE'
bdpt_dict[38]='PUBICREGION'
bdpt_dict[75]='HEAD'
bdpt_dict[76]='FACE'
bdpt_dict[77]='EYEBALL'
bdpt_dict[78]='UPPERTRUNK(OLD)'
bdpt_dict[79]='LOWERTRUNK'
bdpt_dict[80]='UPPERARM'
bdpt_dict[81]='UPPERLEG'
bdpt_dict[82]='HAND'
bdpt_dict[83]='FOOT'
bdpt_dict[84]='25-50% OF BODY'
bdpt_dict[85]='ALLPARTSBODY'
bdpt_dict[86]='OTHER(OLD)'
bdpt_dict[87]='NOTSTATED/UNK'
bdpt_dict[88]='MOUTH'
bdpt_dict[89]='NECK'
bdpt_dict[90]='LOWERARM(OLD)'
bdpt_dict[91]='LOWERLEG(OLD)'
bdpt_dict[92]='FINGER'
bdpt_dict[93]='TOE'
bdpt_dict[94]='EAR'


In [7]:
data['body_string']=data['Body_Part'].map(bdpt_dict)
data['Narrative_LLM']=data["activity_at_injury"].astype(str) + ' '+data["injury_mechanism"].astype(str)+ ' ' + data["object_involved"].astype(str)

In [8]:
# data['Narrative']=data['Narrative'].str.upper()
data[['Narrative']]

,Narrative
0,14YOM REPORTS HE FELL 1 WEEK AND COMPLAINS OF ...
1,A 28YOM BENT TO PICK UP CRATE AT HOME TO ED WI...
2,35YOMRIDING ON MOUNTAIN BIKE PRACTICING FELL D...
3,14 MONTH OLD FEMALE ABRASION FOR NOSE AND FORE...
4,4YR M PLAYING WITH TOY KITCHEN APPLIANCE AND ...
...,...
352046,11 YOM HELPING LOAD BICYCLE IN VAN CUT HEAD DX...
352047,47YOF PUSHING DOWN FOOD IN BLENDER MAKING A SM...
352048,2 YO MALE FELL DOWN STEPS DX HEAD INJURY
352049,38YOM WAS RIDING A BIKE AND FELL WRIST INJURY


In [9]:
replace_list=['ANKLE', 'ARM', 'BODY_PART', 'CHEST', 'CONTUSION', 'CUT', 'EAR', 'ELBOW', 'EYE', 'FACE', 'FINGER', 'FOOT', 'FOREHEAD', 'FRACTURE', 'FX', 'HAND', 'HEAD', 'HIP', 'KNEE', 'LAC', 'LACERATION', 'LEG', 'LOC', 'LOSE', 'NECK', 'PAIN', 'SHOULDER', 
'SPRAIN', 'STRAIN', 'SWELL', 'THUMB', 'TOE', 'WRIST','ABRASION', 'ACHE', 'BREAK', 'BURN', 'CHIN', 'CUT', 'ER', 'FRACTURE', 'FX', 'HIT', 'INJURY', 'LACERATION', 'LIP', 'LOSE', 'LOC', 'MOUTH', 'NOSE', 'PAIN', 'RIB', 'SCALP', 'SPRAIN', 'STRAIN', 'SWELL', 'TOE', 'TWIST', 'WRIST']

for i in replace_list:
  data['Narrative'] = data['Narrative'].str.replace(i, '')
  data['Narrative_LLM']= data['Narrative_LLM'].str.replace(i, '')


data['Narrative'] = data['Narrative'].str.replace('YOM', '')
data['Narrative'] = data['Narrative'].str.replace('YOF', '')
data['Narrative'] = data['Narrative'].str.replace('YR', '')
data['Narrative'] = data['Narrative'].str.replace('OLD', '')
data['Narrative'] = data['Narrative'].str.replace('MALE', '')
data['Narrative'] = data['Narrative'].str.replace('FEMALE', '')
data['Narrative'] = data['Narrative'].str.replace(' YO ', '')
data['Narrative'] = data['Narrative'].str.replace('YO ', '')
data['Narrative'] = data['Narrative'].str.replace(' F ', '')
data['Narrative'] = data['Narrative'].str.replace('YF', '')
data['Narrative'] = data['Narrative'].str.replace(' M ', '')
data['Narrative'] = data['Narrative'].str.replace('ACCIDENTALLY','')
data['Narrative'] = data['Narrative'].str.replace('AGO', '')
data['Narrative'] = data['Narrative'].str.replace('TODAY', '')
data['Narrative'] = data['Narrative'].str.replace('YESTERDAY', '')
data['Narrative'] = data['Narrative'].str.replace('PATIENT', '')
data['Narrative'] = data['Narrative'].str.replace(' PT ', '')
data['Narrative'] = data['Narrative'].str.replace('INJURY', '')
data['Narrative'] = data['Narrative'].str.replace('REPORT', '')
data['Narrative'] = data['Narrative'].str.replace('HURT', '')
data['Narrative'] = data['Narrative'].str.replace('INJ', '')
data['Narrative'] = data['Narrative'].str.replace('FELL', 'FALL')
data['Narrative'] = data['Narrative'].str.replace('INJURE', '')
data['Narrative'] = data['Narrative'].str.replace('JURED', '')
data['Narrative'] = data['Narrative'].str.replace('URED', '')
data['Narrative'] = data['Narrative'].str.replace(' ED', '')
data['Narrative'] = data['Narrative'].str.replace(' RT ', '')
data['Narrative'] = data['Narrative'].str.replace(' LT ', '')

data['Narrative_LLM'] = data['Narrative_LLM'].str.replace('fell', 'fall')


In [10]:
data_core=data.copy() #[core_col]
data_core.shape

(329101, 420)

In [ ]:
data_core=data_core.sample(frac=1,random_state=42).reset_index(drop=True)
# data_test=data_core.head(21000).reset_index(drop=True)
# data_fit=data_core.tail(191347).reset_index(drop=True)
total_rows, n_columns = data.shape

test_size = int(total_rows * 0.2)
train_size = total_rows - test_size

print(f"Original dataset shape: ({total_rows}, {n_columns})")
print(f"Train set size: {train_size} rows")
print(f"Test set size: {test_size} rows")

data_fit = data_core.tail(train_size).reset_index(drop=True)
data_test = data_core.head(test_size).reset_index(drop=True)

data_ready = data_fit
data_ready_test = data_test

print(data_fit.shape)
print(data_test.shape)

Original dataset shape: (329101, 420)
Train set size: 263281 rows
Test set size: 65820 rows
(65820, 420)
(263281, 420)


In [13]:
drop_list=['Narrative','Unnamed: 0',
 'CPSC_Case_Number',
 'Treatment_Date','Race',
 'Other_Race',
 'Hispanic','Diagnosis',
 'Other_Diagnosis',
 'Body_Part_2',
 'Diagnosis_2',
 'Other_Diagnosis_2',
 'Disposition','Fire_Involvement','Product_2',
 'Product_3',
 'Alcohol',
 'Drug','Stratum',
 'PSU',
 'Weight',
 'Year',
 'Month',
 'Day',
 'activity_at_injury',
 'injury_mechanism',
 'object_involved',
 # '#_prod',
 'Disposition_recode',
 'Disposition_recode_2',
 'body_string',
 'Narrative_LLM',
 # 'Processed_Narrative',
 # 'Processed_Narrative_LLM'
          ]

drop_list_test=['Narrative','Unnamed: 0',
 'CPSC_Case_Number',
 'Treatment_Date','Race',
 'Other_Race',
 'Hispanic','Diagnosis',
 'Other_Diagnosis',
 'Body_Part_2',
 'Diagnosis_2',
 'Other_Diagnosis_2',
 'Disposition','Fire_Involvement','Product_2',
 'Product_3',
 'Alcohol',
 'Drug','Stratum',
 'PSU',
 'Weight',
 'Year',
 'Month',
 'Day',
 'activity_at_injury',
 'injury_mechanism',
 'object_involved',
 # '#_prod',
 'Disposition_recode',
 'Disposition_recode_2',
 'body_string',
 # 'Narrative_LLM'
]
 

In [14]:
X = data_ready.drop(drop_list, axis=1) 
y = data_ready['Disposition_recode_2']

print(list(X.columns))

['Age', 'Sex', 'Body_Part', 'Location', 'Product_1', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', 

In [15]:
X.head(10)

,Age,Sex,Body_Part,Location,Product_1,0,1,2,3,4,...,374,375,376,377,378,379,380,381,382,383
0,204,1,36,0,1842,-0.067663,-0.024488,0.011827,-0.013071,-0.044277,...,0.040410,-0.015594,-0.004642,-0.077131,-0.051426,0.001964,-0.075250,-0.015877,-0.022512,0.039195
1,222,1,76,1,4076,-0.013148,0.017099,0.007439,0.021068,-0.014327,...,0.022595,-0.020090,-0.013995,-0.022860,-0.027769,-0.007652,-0.079290,-0.040927,-0.043041,0.010926
2,217,2,75,1,1893,-0.008524,-0.016457,0.038609,0.023419,0.019448,...,0.009474,-0.014518,-0.044684,-0.009650,0.014781,0.028701,-0.071443,-0.025588,-0.040304,0.026923
3,17,1,82,1,836,-0.040323,0.035681,0.038601,-0.001048,0.006665,...,-0.078399,0.040548,-0.000696,-0.041322,0.004424,0.022225,-0.007381,-0.033687,0.021462,0.061621
4,14,1,92,0,1205,-0.055403,-0.056899,0.049341,-0.055997,0.021470,...,0.011134,0.000698,-0.023955,0.023095,-0.017780,0.034502,-0.017278,-0.000838,-0.020978,0.024892
5,18,1,94,9,1270,-0.092189,-0.017336,-0.003985,-0.030710,-0.027104,...,0.002706,-0.038001,0.025524,0.015808,-0.038224,0.032623,-0.031966,-0.095354,0.016343,0.039688
6,47,1,77,1,899,-0.047454,0.023472,0.005736,-0.024613,0.025865,...,0.032333,-0.065269,0.027559,0.025794,0.015561,-0.011675,-0.054703,-0.011628,0.029805,0.033326
7,220,1,75,1,127,-0.025776,0.012047,0.015595,0.026953,0.021451,...,0.059037,-0.036648,-0.001764,-0.028622,-0.043796,0.053549,-0.052174,-0.058245,-0.012008,0.008778
8,5,1,76,1,611,0.026740,-0.006377,0.080827,0.012803,-0.022719,...,0.051690,-0.063737,0.008211,0.022680,0.000173,0.000723,-0.045505,0.015406,-0.058994,0.019330
9,48,2,37,1,1141,-0.068257,-0.040574,0.051002,0.008619,0.010276,...,0.019926,-0.041784,-0.014793,-0.092890,-0.035961,0.027947,-0.076408,-0.011975,-0.116890,0.026060


In [16]:
from sklearn.metrics import f1_score, classification_report, accuracy_score, roc_auc_score
from scipy.stats import mode
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import numpy as np
from sklearn.neural_network import MLPClassifier

# Separate features (X) and target variable (y)
X = data_ready.drop(drop_list, axis=1) 
y = data_ready['Disposition_recode_2']

# Encode target variable if it's categorical
le = LabelEncoder()
y = le.fit_transform(y)

# Split data into training and testing sets
X_train, _, y_train, _ = train_test_split(X, y, test_size=0.2, random_state=42)

X_test=data_ready_test.drop(drop_list_test, axis=1)
y_test=data_ready_test['Disposition_recode_2']
y_test = le.fit_transform(y_test)

# Create a scaler for numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.select_dtypes(include=['number']))

with open('X_scaler_legacy.pkl', 'wb') as file:
    pickle.dump(scaler, file)

X_test_scaled = scaler.transform(X_test.select_dtypes(include=['number']))

# Convert scaled features back to DataFrame
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.select_dtypes(include=['number']).columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.select_dtypes(include=['number']).columns)

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_scaled_df, y_train)

# Combine scaled numerical features with categorical features
X_train_final = X_resampled.copy()
X_test_final = X_test_scaled_df.copy()
y_train_final=y_resampled.copy()


# Define models and their parameter grids for hyperparameter tuning
models = {
    #'KNN': (KNeighborsClassifier(), {'knn__n_neighbors': [3, 5, 7]}),
    #'Random_Forest': (RandomForestClassifier(), {'random_forest__n_estimators': [100, 200], 'random_forest__max_depth': [4, 8]}),
    'XGBoost': (XGBClassifier(
                    objective='binary:logistic',
                    # device = "cuda"
                ), 
           {'xgboost__learning_rate': [0.1, 0.01, 0.001],            # Step size shrinkage to prevent overfitting
            'xgboost__max_depth': [3, 5, 7],                         # Maximum depth of each tree
            #'xgboost__n_estimators': [100, 200, 300],                # Number of boosting rounds
            #'xgboost__min_child_weight': [1, 3, 5],                  # Minimum sum of instance weight needed in a child
            #'xgboost__subsample': [0.6, 0.8,0.9],                   # Subsample ratio of the training instances
            #'xgboost__colsample_bytree': [0.6, 0.8, 0.9],            # Subsample ratio of columns when constructing each tree
            #'xgboost__gamma': [0.1, 0.5, 1,3],                     # Minimum loss reduction required for a split
            #'xgboost__reg_alpha': [0.01, 0.1,0.5,1],                    # L1 regularization term on weights
            #'xgboost__reg_lambda': [1, 1.5, 2],                      # L2 regularization term on weights
            'xgboost__scale_pos_weight': [1, 5, 10]
            }),
    #'Logistic_Regression': (LogisticRegression(solver='liblinear'), {'logistic_regression__C': [0.1, 1, 10]}),
    #'SVM': (SVC(probability=True), {'svm__C': [0.1, 1, 10], 'svm__kernel': ['linear', 'rbf']}),
    #'NN': (MLPClassifier(max_iter=1000), {'nn__hidden_layer_sizes': [(10,), (50,), (100,)], 'nn__activation': ['relu', 'tanh']}),
}

# Initialize list to store predictions from each model

ensemble_predictions = []
ensemble_probabilities = []

for model_name, (model, param_grid) in models.items():
    print(f"Training {model_name}...")

    # Create a pipeline for preprocessing and model training
    pipeline = Pipeline([
        (model_name.lower(), model)
    ])

    # Perform hyperparameter tuning using GridSearchCV with AUC score
    grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='precision',n_jobs=8)
    grid_search.fit(X_train_final, y_train_final)

    # Get the best model from the grid search
    best_model = grid_search.best_estimator_

    # Make predictions and predict probabilities on the test set
    y_pred = best_model.predict(X_test_final)
    y_prob = best_model.predict_proba(X_test_final)[:, 1]  # Probability for the positive class
    ensemble_predictions.append(y_pred)
    ensemble_probabilities.append(y_prob)

    # Evaluate the model using accuracy, F1-score, and AUC
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)
    print(f"{model_name} Accuracy: {accuracy}")
    print(f"{model_name} F1 Score: {f1}")
    print(f"{model_name} AUC: {auc}")
    print(classification_report(y_test, y_pred))

# Implement ensemble method (majority voting for predictions, average for probabilities)
ensemble_predictions = np.array(ensemble_predictions)
ensemble_pred_final = mode(ensemble_predictions, axis=0)[0].flatten()

# Average the probabilities for the ensemble AUC
ensemble_probabilities = np.mean(ensemble_probabilities, axis=0)

# Evaluate the ensemble model using accuracy, F1-score, and AUC
ensemble_accuracy = accuracy_score(y_test, ensemble_pred_final)
ensemble_f1 = f1_score(y_test, ensemble_pred_final)
ensemble_auc = roc_auc_score(y_test, ensemble_probabilities)
print("Ensemble Accuracy:", ensemble_accuracy)
print("Ensemble F1 Score:", ensemble_f1)
print("Ensemble AUC:", ensemble_auc)
print(classification_report(y_test, ensemble_pred_final))

with open('xgboost_tfidf_LLM_legacy.pkl', 'wb') as file:
    pickle.dump(best_model, file)

Training XGBoost...
XGBoost Accuracy: 0.887724096019447
XGBoost F1 Score: 0.45581737849779086
XGBoost AUC: 0.8530252097160516
              precision    recall  f1-score   support

           0       0.94      0.93      0.94     59231
           1       0.44      0.47      0.46      6589

    accuracy                           0.89     65820
   macro avg       0.69      0.70      0.70     65820
weighted avg       0.89      0.89      0.89     65820

Ensemble Accuracy: 0.887724096019447
Ensemble F1 Score: 0.45581737849779086
Ensemble AUC: 0.8530252097160516
              precision    recall  f1-score   support

           0       0.94      0.93      0.94     59231
           1       0.44      0.47      0.46      6589

    accuracy                           0.89     65820
   macro avg       0.69      0.70      0.70     65820
weighted avg       0.89      0.89      0.89     65820



In [17]:
X_test_final.head(5)


,Age,Sex,Body_Part,Location,Product_1,0,1,2,3,4,...,374,375,376,377,378,379,380,381,382,383
0,-0.569565,-0.918442,0.798008,-0.759005,-0.630488,-0.028556,-0.210244,0.653458,0.422594,-0.552273,...,0.096302,-1.457560,-0.940383,-0.431679,0.861529,0.727539,0.584592,1.815091,-0.930339,0.738120
1,-0.027629,1.088217,1.185319,-0.759005,-1.235149,-0.100331,0.389565,-1.184388,-0.859979,0.961187,...,-1.869818,0.530019,0.660291,-1.537514,3.054271,0.232845,1.035868,0.902119,0.822908,0.383248
2,0.847805,-0.918442,0.453732,-0.448876,-0.244833,-2.157367,1.251366,0.606688,0.269229,-0.765541,...,0.062612,-0.934253,1.410312,-0.870976,-0.050991,0.939831,0.353818,0.515875,-0.540346,-1.661214
3,-0.882220,1.088217,-1.396753,-0.448876,-0.244833,-0.684667,-0.750389,0.382090,1.209154,-1.316196,...,1.504670,0.043557,0.526457,-0.385281,-0.896596,-0.220806,-0.548746,0.225890,-1.170562,-0.536523
4,-0.673783,-0.918442,-1.181581,2.032149,-0.684318,-0.800479,0.115145,-0.648785,-1.176367,0.672627,...,0.131281,0.345309,-1.039521,0.629269,-0.552373,0.324872,0.943851,0.612526,0.564084,1.127774


In [18]:
scaled_body_parts = X_test_final['Body_Part'].unique()
scaled_body_parts.sort()
scaled_body_parts

array([-1.48282232, -1.43978778, -1.39675325, -1.35371871, -1.31068418,
       -1.26764965, -1.22461511, -1.18158058, -1.13854604,  0.45373172,
        0.49676626,  0.53980079,  0.62586986,  0.66890439,  0.71193893,
        0.75497346,  0.79800799,  1.01318066,  1.0562152 ,  1.1853188 ,
        1.22835334,  1.27138787])

In [19]:
for index, row in X_test_final[:10].iterrows():
    prob = []
    for body_part in scaled_body_parts:
        current_sample = row.copy()
        current_sample['Body_Part'] = body_part
        # print(current_sample)
        prob.append(best_model.predict_proba([current_sample])[0][0])
    prob.sort()
    print(prob[0], prob[-1]) # min max

0.7774098 0.97443384
0.7871166 0.9850918
0.10643506 0.78080165
0.37823087 0.9674263
0.6412952 0.9589059
0.570647 0.97149336
0.114775 0.62799233
0.7816659 0.9750805
0.29187936 0.89629775
0.8799956 0.98634934
